# Morphometric plots

This notebook includes code to generate figures related to spinal cord morphometrics, for participants included in the PAM50 template. 

The scripts included in this notebook are inspired by this pipeline : https://github.com/sct-pipeline/pediatric-data-analysis/blob/main/results/plots/morphometrics.ipynb

In [95]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import yaml
import plotly.graph_objects as go
from statsmodels.stats.multitest import multipletests
from plotly.subplots import make_subplots
import statsmodels.formula.api as smf
import re

### Load config file to get path to dataset 

In [96]:
# Load config file
with open('../config/config.yaml' , 'r') as file:
    config = yaml.safe_load(file)

# Get data path from config file
path_data = config['path_data']

### Get the `participants.tsv` file from the dataset

In [97]:
# Get path to participants.tsv file
participants_tsv = pd.read_csv(os.path.join(path_data, 'participants.tsv'), sep='\t')
participants_tsv

,participant_id,source_id,species,age,sex,height (cm),weight (kg),pathology,institution,stenosis
0,sub-amuAL,amu_AL_T045,homo sapiens,25.0,M,180.0,70.0,HC,amu,NaN
1,sub-amuALT,amu_ALT,homo sapiens,34.0,M,176.0,72.0,HC,amu,T7-T8
2,sub-amuAM,amu_AM_T094,homo sapiens,28.0,M,192.0,70.0,HC,amu,"C3-C4,C4-C5"
3,sub-amuAP,amu_AP_T077,homo sapiens,22.0,F,160.0,53.0,HC,amu,NaN
4,sub-amuCR,amu_CR_T020b,homo sapiens,25.0,M,179.0,67.0,HC,amu,NaN
...,...,...,...,...,...,...,...,...,...,...
57,sub-unfPain004,unf_d_sp_pain_pilot4,homo sapiens,22.0,M,NaN,71.0,HC,unf,"T3-T4,T5-T6"
58,sub-unfPain005,unf_d_sp_pain_pilot7,homo sapiens,21.0,M,NaN,68.0,HC,unf,NaN
59,sub-unfPain006,unf_p_sp_pain_pilot2,homo sapiens,31.0,M,180.0,75.0,HC,unf,T8-T9
60,sub-unfSCT001,unf_sct_001,homo sapiens,NaN,NaN,NaN,NaN,NaN,unf,NaN


## Dataframe of subjects included in the PAM50

The following dataframe contains only the subjects from the whole-spine dataset that were included in the PAM50 template (N=50)

In [98]:
# Include only participants included in the `include_list` key from the `config.yml` file
include_list = config['include_list']
PAM50_participants_tsv = participants_tsv[participants_tsv['participant_id'].isin(include_list)]
PAM50_participants_tsv.reset_index(drop=True, inplace=True) # Reset index (0 to 50)
PAM50_participants_tsv

,participant_id,source_id,species,age,sex,height (cm),weight (kg),pathology,institution,stenosis
0,sub-amuAL,amu_AL_T045,homo sapiens,25.0,M,180.0,70.0,HC,amu,NaN
1,sub-amuALT,amu_ALT,homo sapiens,34.0,M,176.0,72.0,HC,amu,T7-T8
2,sub-amuAM,amu_AM_T094,homo sapiens,28.0,M,192.0,70.0,HC,amu,"C3-C4,C4-C5"
3,sub-amuED,amu_ED,homo sapiens,30.0,F,168.0,50.0,HC,amu,NaN
4,sub-amuFR,amu_FR_T080,homo sapiens,NaN,NaN,NaN,NaN,HC,amu,NaN
5,sub-amuGB,amu_GB_T083,homo sapiens,NaN,NaN,NaN,NaN,HC,amu,T9-T10
6,sub-amuHB,amu_HB_T093,homo sapiens,30.0,M,183.0,72.0,HC,amu,NaN
7,sub-amuJD,amu_JD,homo sapiens,33.0,M,188.0,72.0,HC,amu,NaN
8,sub-amuJW,amu_JW,homo sapiens,26.0,M,173.0,77.0,HC,amu,NaN
9,sub-amuLJ,amu_LJ_T047,homo sapiens,22.0,F,175.0,62.0,HC,amu,T4-T5


## Plot demographics

This function plots the age and sex distribution of the subjects included in a pipeline analysis, according to the include list generated above. 

In [99]:
def plot_demographics(df):
    """
    This function plots the demographic information of participants, given a dataframe with the list of subjects to include in the analysis.
    """

    # Sort by sex
    df_M = df[df['sex'] == 'M']
    df_F = df[df['sex'] == 'F']

    # Round down age to nearest month 
    df['age'] = np.floor(df['age']) 

    # Create subplot
    fig = make_subplots(rows=1, cols=1)

    # Add histogram for female subjects
    fig.add_trace(go.Histogram(
        x=df_F['age'], 
        name='F', 
        marker=dict(color= "#D19D88"),
        opacity=1.0,
        legendgroup='F',
        ),
        row=1, col=1
    )

    # Add histogram for male subjects
    fig.add_trace(go.Histogram(
        x=df_M['age'], 
        name='M', 
        
        marker=dict(color="#5C8EA1"),
        opacity=1.0,
        legendgroup='M',
        ), 
        row=1, col=1
    )

    # Define age tick range
    # tick_vals = list(range(20, 60)) 

    # Update layout
    fig.update_layout(
        width=900,
        height=500,
        font=dict(family='Arial', size=20, color='black'), 
        legend=dict(
            orientation="h", 
            yanchor="bottom", 
            y=1.0, 
            xanchor="center",  
            x=0.5,
        ),
        xaxis=dict(
            range=[20, 60],  # Set x-axis range from 6 to 17
        ),
        plot_bgcolor='white',
        barmode='stack',
        bargap=0.3,  
        xaxis_title='Age (years)',
        xaxis_title_font=dict(family='Arial', size=20, weight='bold'),
        yaxis_title='Number of Subjects',
        yaxis_title_font=dict(family='Arial', size=20, weight='bold'),
        xaxis_title_standoff=50, 
    )

    fig.update_xaxes(
        tickmode='array',
        # tickvals=tick_vals,
        showgrid=False,
        gridwidth=1
    )

    fig.update_yaxes(
        showgrid=True,             # Horizontal grid lines
        gridcolor='lightgrey',
        gridwidth=1
    )

    # Set bin size to 1 year
    fig.update_traces(xbins=dict(size=1))

    fig.show()

In [100]:
# Plot demographics for included participants in the PAM50
plot_demographics(PAM50_participants_tsv)

/tmp/ipykernel_19837/2805166897.py:11: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



### Path to morphometrics

In [101]:
# Path to morphometric
morphometric_results_dir = '../results/morphometrics/'

# Path to participants.tsv
participants_info_path = os.path.join(path_data, 'participants.tsv')

Load the CSV files containing morphometric values of each subject and add to a single dataframe

In [102]:
# Initialize an empty DataFrame to store morphometric values
morphometrics_df = pd.DataFrame()

# Load the CSV files containing morphometric values of each subject and add to a single dataframe
for file in os.listdir(morphometric_results_dir):
    if file.endswith('_interpolated_morphometrics.csv'):
        df = pd.read_csv(os.path.join(morphometric_results_dir, file))
        morphometrics_df = pd.concat([morphometrics_df, df], ignore_index=True)

# Remove age and sex columns if they exist
if 'age' in morphometrics_df.columns:
    morphometrics_df = morphometrics_df.drop(columns=['age'])
if 'sex' in morphometrics_df.columns:
    morphometrics_df = morphometrics_df.drop(columns=['sex'])

In [103]:
morphometrics_df

,subject,VertLevel,DistancePMJ,CSA,AP_diameter,RL_diameter,eccentricity,solidity
0,sub-amuFR,1.0,26.880,95.441153,10.006485,12.116241,0.563531,0.973235
1,sub-amuFR,1.1,28.888,87.975606,9.411010,11.918463,0.613058,0.969622
2,sub-amuFR,1.2,30.896,81.656190,8.908212,11.696705,0.648008,0.960672
3,sub-amuFR,1.3,32.904,79.308773,8.690918,11.642384,0.664378,0.967786
4,sub-amuFR,1.4,34.912,76.276527,8.283358,11.717558,0.707207,0.961208
...,...,...,...,...,...,...,...,...
1392,sub-amuAM,20.6,NaN,56.236594,6.926026,9.910782,0.674377,0.970105
1393,sub-amuAM,20.7,NaN,56.236594,6.926026,9.910782,0.674377,0.970105
1394,sub-amuAM,20.8,NaN,56.236594,6.926026,9.910782,0.674377,0.970105
1395,sub-amuAM,20.9,NaN,56.236594,6.926026,9.910782,0.674377,0.970105


In [104]:
# Add sex and age columns to the DataFrame (using participants.tsv)
participants_info = pd.read_csv(participants_info_path, sep='\t')
participants_info = participants_info.rename(columns={'participant_id': 'subject'})
participants_info

participants_info = participants_info[['subject', 'age', 'sex']]
morphometrics_df = morphometrics_df.merge(participants_info, on='subject', how='left')

In [105]:
morphometrics_df

,subject,VertLevel,DistancePMJ,CSA,AP_diameter,RL_diameter,eccentricity,solidity,age,sex
0,sub-amuFR,1.0,26.880,95.441153,10.006485,12.116241,0.563531,0.973235,NaN,NaN
1,sub-amuFR,1.1,28.888,87.975606,9.411010,11.918463,0.613058,0.969622,NaN,NaN
2,sub-amuFR,1.2,30.896,81.656190,8.908212,11.696705,0.648008,0.960672,NaN,NaN
3,sub-amuFR,1.3,32.904,79.308773,8.690918,11.642384,0.664378,0.967786,NaN,NaN
4,sub-amuFR,1.4,34.912,76.276527,8.283358,11.717558,0.707207,0.961208,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
1392,sub-amuAM,20.6,NaN,56.236594,6.926026,9.910782,0.674377,0.970105,28.0,M
1393,sub-amuAM,20.7,NaN,56.236594,6.926026,9.910782,0.674377,0.970105,28.0,M
1394,sub-amuAM,20.8,NaN,56.236594,6.926026,9.910782,0.674377,0.970105,28.0,M
1395,sub-amuAM,20.9,NaN,56.236594,6.926026,9.910782,0.674377,0.970105,28.0,M


Function to generate a regression plot for a given morphometric (with respect to age) :

In [106]:
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
import webbrowser
import os

def plot_morphometrics(df, morphometric_name_in_csv, morphometric_display_name):

    fig = go.Figure()

    # Get unique subjects
    unique_subjects = sorted(df['subject'].unique())

    # Create a color gradient
    colorscale = px.colors.sequential.Viridis
    num_colors = len(unique_subjects)
    color_gradient = px.colors.sample_colorscale(
        colorscale,
        [i / max(num_colors - 1, 1) for i in range(num_colors)]
    )

    color_map = {
        subject: color_gradient[i]
        for i, subject in enumerate(unique_subjects)
    }

    # Loop over subjects
    for subject in unique_subjects:

        subject_df = df[df['subject'] == subject].sort_values('VertLevel')

        x = subject_df['VertLevel']
        y = subject_df[morphometric_name_in_csv]

        fig.add_trace(go.Scatter(
            x=x,
            y=y,
            mode='lines',
            name=f'{subject}',
            line=dict(color=color_map[subject], width=2),
            opacity=0.8
        ))

    # Layout
    fig.update_layout(
        width=1000,
        height=600,
        xaxis_title='Vertebral Level',
        yaxis_title=morphometric_display_name,
        template='plotly_white',
        legend_title='Subjects'
    )

    # Anatomical label mapping
    vertebral_label_map = {
        1.0: "C1", 2.0: "C2", 3.0: "C3", 4.0: "C4", 5.0: "C5",
        6.0: "C6", 7.0: "C7", 8.0: "T1", 9.0: "T2", 10.0: "T3",
        11.0: "T4", 12.0: "T5", 13.0: "T6", 14.0: "T7", 15.0: "T8",
        16.0: "T9", 17.0: "T10", 18.0: "T11", 19.0: "T12", 20.0: "L1"
    }

    all_vert_levels = sorted(df['VertLevel'].dropna().unique())
    integer_vert_levels = [
        v for v in all_vert_levels
        if float(v).is_integer() and v in vertebral_label_map
    ]

    ticktexts = [vertebral_label_map[v] for v in integer_vert_levels]

    fig.update_xaxes(
        tickmode='array',
        tickvals=integer_vert_levels,
        ticktext=ticktexts
    )

    # Save
    save_path = os.path.join('..', 'figures',
                             f'{morphometric_name_in_csv}_vs_Vert_Levels.html')

    fig.write_html(save_path)
    webbrowser.open(f'file://{os.path.abspath(save_path)}')
    fig.show()


### 1. Spinal cord cross-sectional area (CSA)

In [107]:
plot_morphometrics(morphometrics_df, 'CSA','CSA (mm²)')

### 2. Antero-posterior (AP) diameter, right-left (RL) diameter

In [108]:
plot_morphometrics(morphometrics_df, 'AP_diameter', 'AP diameter (mm)')
plot_morphometrics(morphometrics_df, 'RL_diameter', 'RL diameter (mm)')

### 3. Eccentricity and solidity

In [109]:
plot_morphometrics(morphometrics_df, 'eccentricity', 'Eccentricity (mm)')